# Ingestão Bronze - Cotação Histórica

Este notebook realiza a ingestão da cotação histórica a partir dos dados do arquivo de cotação histórica da B3 e grava os dados em formato Delta na camada **Bronze**.

**Origem dos dados:**
- Arquivo: `D:\Projetos\Jornada_financas_pessoais\data\source\COTAHIST_AYYYY.TXT`

**Destino:**
- Caminho Delta: `D:\Projetos\Jornada_financas_pessoais\data\delta\bronze\`
- Tabela Delta: `bronze.raw_cotahist`

## Imports

In [1]:
from spark_config import init_spark
from pyspark.sql import functions as F
import glob
import os

## Start Spark Session

In [2]:
spark = init_spark("Leitura de arquivo de formato fixo")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Variables

In [3]:
# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"

# Define as posições iniciais e finais de cada campo
positions = {"tipo_registro": (1, 2),
               "data_pregao": (3, 10),
               "codigo_bdi": (11, 12),
               "codigo_negociacao": (13, 24),
               "tipo_mercado": (25, 27),
               "nome_resumido_empresa": (28, 39),
               "especificacao_papel": (40, 49),
               "prazo_dias_mercado": (50, 52),
               "moeda_referencia": (53, 56),
               "preco_abertura_papel": (57, 69),
               "preco_maximo_papel": (70, 82),
               "preco_minimo_papel": (83, 95),
               "preco_medio_papel": (96, 108),
               "preco_ultimo_negocio": (109, 121),
               "preco_melhor_oferta_compra": (122, 134),
               "preco_melhor_oferta_venda": (135, 147),
               "numero_negocios_efetuados": (148, 152),
               "quantidade_total_titulos": (153, 170),
               "volume_total_titulos": (171, 188),
               "preco_exercicio_opcoes": (189, 201),
               "indicador_correcao_precos": (202, 202),
               "data_vencimento_opcoes": (203, 210),
               "fator_cotacao_papel": (211, 217),
               "preco_exercicio_pontos": (218, 230),
               "codigo_papel_sistema": (231, 242),
               "numero_distribuicao_papel": (243, 245)
			   }
        

## Read File

In [4]:
# Busca todos os arquivos que começam com 'COTAHIST' no diretório
file_list = glob.glob(f'{source_path}/COTAHIST*.TXT')

# Lê todos os arquivos encontrados
df = spark.read.text(file_list)

# Lê todos os arquivos encontrados e adiciona uma coluna com o nome do arquivo
df = (
    spark.read.text(file_list)
    .withColumn("nome_arquivo", F.regexp_extract(F.input_file_name(), r'[^/\\]+$', 0))  # Somente o nome do arquivo
)

print(f"Arquivos encontrados: {len(file_list)}")
for f in file_list:
    print(f"  - {os.path.basename(f)}")

Arquivos encontrados: 1
  - COTAHIST_A2025.TXT


## Transformations

In [5]:
# Extrai os campos em formato fixo usando a função substring e trim. Mantém o nome do arquivo
df = df.select(
    *[F.trim(F.substring(F.col("value"), start, end - start + 1)).alias(field)
      for field, (start, end) in positions.items()],
    F.col("nome_arquivo") 
)

df = df.filter(F.col("tipo_registro") == "01")

print(df.count(), "registros lidos")

3094053 registros lidos


## Save Data Frame

In [6]:
# grava os dados em formato Delta (substitui os anteriores)
df.write.format("delta").mode("overwrite").option("overwriteSchema", "false").save(delta_path_cotahist)

print(f"[SUCESSO] DataFrame gravado no Delta Lake em: {delta_path_cotahist}")

[SUCESSO] DataFrame gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cotahist


## Stop Spark Session

In [7]:
# Encerra a SparkSession
spark.stop()